# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [51]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from decouple import config
g_key = config('g_key')



In [52]:
#use your own key by commenting out my g_key above and uncomment the code below. When prompted at runtime, 
# Paste your key and run
#g_key = input('your key:')

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [53]:
df = pd.read_csv('output_data/cities.csv')

In [54]:
df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Albany,100,US,1601646826,90,42.60,-73.97,53.01,6.24
1,Nouadhibou,0,MR,1601646827,43,20.93,-17.03,87.80,26.40
2,Caruray,93,PH,1601646827,77,10.31,119.01,82.15,2.04
3,Asău,82,RO,1601646827,73,46.43,26.40,64.99,4.47
4,Muzhi,100,RU,1601646827,77,65.37,64.67,48.34,10.80
...,...,...,...,...,...,...,...,...,...
529,Kulhudhuffushi,49,MV,1601646968,75,6.62,73.07,82.20,10.98
530,Huilong,84,CN,1601646969,82,31.81,121.66,73.99,1.99
531,Linguère,0,SN,1601646969,13,15.40,-15.12,107.01,12.03
532,Kuching,40,MY,1601646862,100,1.55,110.33,77.00,3.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [55]:
locations = df[['Lat','Lng']]
weights = df['Humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [56]:
narrowed_df = df.loc[(df['Humidity'] < 70) & (df['Max Temp'].between(75,90)) & (df['Cloudiness'] < 45)]

In [65]:
narrowed_df.reset_index(inplace = True, drop=True)
#narrowed_df
count = narrowed_df['City'].count()
print(f'{count} cities meet weather conditions')

38 cities meet weather conditions


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [109]:
hotel_df = narrowed_df.assign(like=True).drop('like',axis=1)

In [110]:
hotel_df['Hotel Name'] = ""

In [111]:
hotel_df.head(5)

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Nouadhibou,0,MR,1601646827,43,20.93,-17.03,87.80,26.40,
1,Kinshasa,40,CD,1601646832,49,-4.32,15.32,89.60,4.70,
2,Castro,1,BR,1601646836,29,-24.79,-50.01,89.91,12.46,
3,Ponta do Sol,40,PT,1601646837,57,32.67,-17.10,75.20,11.41,
4,Santarém,38,BR,1601646845,66,-2.44,-54.71,86.00,6.93,


In [114]:
ecount = 0
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    
    
    params = {
        "radius": 5000,
        "types": "hotel",
        "location":f"{lat},{lng}",
        "key": g_key
    }
    print(f"Checking hotel {index} at {lat},{lng}")
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    name_address = requests.get(base_url, params=params)
    name_address = name_address.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address['results'][1]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping")
        ecount += 1
print('-' * 25)
print('Processing Completed')
print(f'{ecount} locations skipped')
print('-' * 25)

Checking hotel 0 at 20.93,-17.03
Checking hotel 1 at -4.32,15.32
Checking hotel 2 at -24.79,-50.01
Checking hotel 3 at 32.67,-17.1
Checking hotel 4 at -2.44,-54.71
Checking hotel 5 at 36.96,45.39
Checking hotel 6 at -22.97,-42.03
Checking hotel 7 at -8.01,-34.86
Checking hotel 8 at -21.64,-41.05
Checking hotel 9 at 37.38,55.49
Checking hotel 10 at 22.57,59.53
Checking hotel 11 at -20.19,32.62
Checking hotel 12 at 31.61,34.76
Checking hotel 13 at 37.27,28.22
Checking hotel 14 at 18.39,42.45
Checking hotel 15 at -20.78,117.13
Checking hotel 16 at 37.84,29.07
Checking hotel 17 at -20.32,118.57
Checking hotel 18 at 31.35,27.25
Checking hotel 19 at 9.61,6.56
Checking hotel 20 at -25.17,45.05
Missing field/result... skipping
Checking hotel 21 at 36.75,24.43
Checking hotel 22 at 43.39,20.75
Checking hotel 23 at 0.05,37.66
Checking hotel 24 at 11.47,49.87
Checking hotel 25 at 23.5,73.5
Checking hotel 26 at 35.49,23.66
Checking hotel 27 at 26.12,127.67
Checking hotel 28 at -5.2,-35.46
Checking 

In [113]:
hotel_df.head(5)

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Nouadhibou,0,MR,1601646827,43,20.93,-17.03,87.80,26.40,El Medina
1,Kinshasa,40,CD,1601646832,49,-4.32,15.32,89.60,4.70,Flat Hotel Alhdeef
2,Castro,1,BR,1601646836,29,-24.79,-50.01,89.91,12.46,Lojas Colombo
3,Ponta do Sol,40,PT,1601646837,57,32.67,-17.10,75.20,11.41,Hotel do Campo
4,Santarém,38,BR,1601646845,66,-2.44,-54.71,86.00,6.93,Hotel Imperador


In [102]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [118]:
# Add marker layer ontop of heat map

locations = hotel_df[['Lat','Lng']]
weights = hotel_df['Humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig.add_layer(gmaps.marker_layer(locations, info_box_content = hotel_info))
fig
# Display Map

Figure(layout=FigureLayout(height='420px'))